## Learning Black-Scholes Equation for Vanilla Options

In [94]:
import numpy as np
import torch
import csv

In [95]:
dataset = []
with open('../data/black_scholes.csv', newline='\n') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  for row in reader:
    dataset.append([float(row[0]), float(row[1]), float(row[2]),
                    float(row[3]), float(row[4]), float(row[5])])

train_set, test_set = torch.tensor(dataset[:7000]), torch.tensor(dataset[7000:])
train_features, train_labels = train_set[:, :5], train_set[:, 5]
test_features, test_labels = test_set[:, :5], test_set[:, 5]

In [96]:
batch_size = 40
train_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_features, train_labels),
                                         batch_size,
                                         shuffle=True)
test_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_features, test_labels),
                                        batch_size,
                                        shuffle=False)


In [97]:
num_inputs = 5
num_hidden = 100
num_outputs = 1
model = torch.nn.Sequential(torch.nn.Linear(num_inputs, num_hidden),
                            torch.nn.LeakyReLU(),
                            torch.nn.Linear(num_hidden, num_hidden),
                            torch.nn.ELU(),
                            torch.nn.Linear(num_hidden, num_hidden),
                            torch.nn.ReLU(),
                            torch.nn.Linear(num_hidden, num_hidden),
                            torch.nn.ELU(),
                            torch.nn.Linear(num_hidden, num_outputs))

In [98]:
loss = torch.nn.MSELoss()

In [99]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [100]:
num_epochs = 10
for epoch in range(num_epochs):
  for X, y in train_iter:
    l = loss(torch.exp(model(X)), y)

    optimizer.zero_grad()
    l.backward()
    optimizer.step()

  l = loss(torch.exp(model(train_features)), train_labels)
  print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 14.028685
epoch 2, loss 14.032825
epoch 3, loss 14.035039
epoch 4, loss 14.027887
epoch 5, loss 14.027248
epoch 6, loss 14.025537
epoch 7, loss 14.029273
epoch 8, loss 14.024611
epoch 9, loss 14.047000
epoch 10, loss 14.033741


In [101]:
with torch.no_grad():
  l = loss(torch.exp(model(test_features)), test_labels)
  print(f'loss {l:f}')

loss 15.404756


In [102]:
torch.save(model.state_dict(), '../save/black_scholes.ckpt')